# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114424e+02     1.649815e+00
 * time: 0.34057092666625977
     1     1.067579e+01     9.148598e-01
 * time: 2.1680049896240234
     2    -1.197023e+01     9.468281e-01
 * time: 2.8000669479370117
     3    -3.372416e+01     7.446659e-01
 * time: 3.735940933227539
     4    -4.726258e+01     5.578375e-01
 * time: 4.680046081542969
     5    -5.685413e+01     2.219295e-01
 * time: 5.6070921421051025
     6    -5.982652e+01     1.656145e-01
 * time: 6.234701156616211
     7    -6.093817e+01     4.941529e-02
 * time: 6.864378929138184
     8    -6.135277e+01     7.924769e-02
 * time: 7.503515005111694
     9    -6.162512e+01     3.486119e-02
 * time: 8.136078119277954
    10    -6.184081e+01     2.457507e-02
 * time: 8.767789125442505
    11    -6.200968e+01     2.091493e-02
 * time: 9.38324499130249
    12    -6.207396e+01     1.811946e-02
 * time: 10.015526056289673
    13    -6.213991e+01     1.438266e-02
 * time: 10.641155004501343
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557688
    AtomicNonlocal      14.8522664
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336818

    total               -62.261666458538
